In [1]:
"""
When pasting the GeekMailing (GM) list into BGG, you might encounter errors.
This can happen because there is a typo in the list and/or because someone changed the username.
Unfortunately, BGG doesn't tell you which username is causing issues.
This code fetches the GM list and uses the BGG API to validate the usernames.
"""

"\nWhen pasting the GeekMailing (GM) list into BGG, you might encounter errors.\nThis can happen because there is a typo in the list and/or because someone changed the username.\nUnfortunately, BGG doesn't tell you which username is causing issues.\nThis code fetches the GM list and uses the BGG API to validate the usernames.\n"

In [2]:
import requests
import xml.etree.ElementTree as ET
import time
from urllib.parse import quote  # For URL-encoding
from html import unescape  # To handle improperly escaped XML entities

# Step 1: Fetch the usernames from the GitHub raw URL
def fetch_usernames_from_github():
    github_url = "https://raw.githubusercontent.com/DrBewd/BGG_top200_at_2_data_repository/main/GeekMailing_list/usernames"
    response = requests.get(github_url)
    if response.status_code == 200:
        # Extract the first line (usernames) and split by comma
        content = response.text.strip()
        first_line = content.split("\n")[0]  # Get the line with usernames
        return [username.strip() for username in first_line.split(",")]  # Split and strip whitespace
    else:
        print(f"Failed to fetch usernames from GitHub: {response.status_code}")
        return []

# Step 2: Check if a username is valid using the BGG API
def is_valid_username(username, max_retries=3, retry_delay=5):
    encoded_username = quote(username)  # URL-encode the username
    bgg_api_url = f"https://boardgamegeek.com/xmlapi2/user?name={encoded_username}"
    retries = 0
    while retries < max_retries:
        try:
            response = requests.get(bgg_api_url, timeout=10)  # Set a timeout
            if response.status_code == 200:
                # Clean up XML response to handle special characters
                cleaned_response = unescape(response.text)
                root = ET.fromstring(cleaned_response)
                if root.tag == "user":
                    return True
            print(f"API Response for {username}: {response.text}")  # Debugging
        except ET.ParseError as e:
            print(f"XML parsing error for username {username}: {e}")
            return False
        except requests.exceptions.RequestException as e:
            print(f"Error checking username {username}: {e}")
        
        # Retry logic
        retries += 1
        print(f"Retrying {username} ({retries}/{max_retries}) after {retry_delay} seconds...")
        time.sleep(retry_delay)
    return False

# Step 3: Validate usernames and print invalid ones
def find_invalid_usernames(usernames):
    invalid_usernames = []
    for username in usernames:
        print(f"Checking username: {username}")
        if not is_valid_username(username):
            print(f"Invalid username: {username}")
            invalid_usernames.append(username)
        time.sleep(1)  # Add a delay to avoid hitting API rate limits
    return invalid_usernames

# Main function
if __name__ == "__main__":
    usernames = fetch_usernames_from_github()
    if usernames:
        invalid_usernames = find_invalid_usernames(usernames)
        print("\nInvalid Usernames:")
        for invalid in invalid_usernames:
            print(invalid)
    else:
        print("No usernames to check.")


Checking username: 2d20
Checking username: Aiken Drum
Checking username: antonioalg
Checking username: Arah
Checking username: astamatis06
Checking username: Bellecupcake
Checking username: Birdwrong
Checking username: BN322
Checking username: bngabletofly
Checking username: Cybaeus
Checking username: dagopher
Checking username: darkritual88
Checking username: dynahthirst
Checking username: Earless
Checking username: eversilver
Checking username: Ezraar
Checking username: Formersd
Checking username: goudie
Checking username: gregrice23
Checking username: Helen Horselip
Checking username: henrysdad
Checking username: Hymenoptera
Checking username: ilovemariko
Checking username: JainaJ
Checking username: JamesD1222
Checking username: JanMS
Checking username: joqose
Checking username: jroddy27
Checking username: Kaila_X
Checking username: kat_splat
Checking username: KevKevKev
Checking username: kintanakely
Checking username: Kwnanan
Checking username: Lemoneg
Checking username: Lurtz3019